In [22]:
import os
import polars as pl

os.chdir("C:\\Users\\ndgig\\Repositories\\grammatron")
print(os.getcwd())

%load_ext autoreload
%autoreload 2

from grammatron.grammatron import Grammatron

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\ndgig\\Repositories\\grammatron'

In [2]:
splits = {
    "train": "data/train-*-of-*.parquet",
    "validation": "data/validation-00000-of-00001-869c898b519ad725.parquet",
}
docs = (
    pl.read_parquet("hf://datasets/roneneldan/TinyStories/" + splits["validation"])
    .sample(1000, seed=32)
    .to_pandas()
    .squeeze()
    .to_list()
)
docs

['John was walking down the street when he saw a vendor. He was so excited! He wanted to buy something, but his mommy said he had to wait. He watched the vendor with big eyes, but the vendor started to ignore him. John started to feel sad until he saw the modern vendor down the street. The vendor was dressed in bright colors and had so many interesting things to buy. \nJohn ran to the vendor as fast as he could and asked his mommy if he could buy something. His mommy said yes, and John was so happy! He went to the vendor, who gave him a big smile, and he chose a fun toy. \nJohn thanked the vendor, and skipped happily all the way home with his new toy, forgetting all about the vendor who chose to ignore him.',
 "Once upon a time, there was a kind lady named Mrs. Smith. She loved to watch cartoons on her TV. One day, she went to the store to buy some toys for the children in her neighborhood. She was very generous and wanted to make the kids happy.\n\nAt the store, she found a box of puz

In [ ]:
if "ng" in locals():
    del ng
ng = Grammatron("sentence-transformers/paraphrase-MiniLM-L3-v2", device="cuda")
ng

In [15]:
ngrams, ngram_vecs = ng.encode_extract(docs, batch_size=256, amp=False)
ngram_vecs

Encoding:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting:   0%|          | 0/1000 [00:00<?, ?it/s]

[array([[-0.0140322 ,  0.00529452,  0.09398252, ..., -0.12720276,
         -0.18071574,  0.15556256],
        [ 0.2069744 ,  0.17052889,  0.19572538, ..., -0.2820218 ,
         -0.16869394,  0.28693688],
        [ 0.25348058,  0.16271347,  0.22691195, ..., -0.3728949 ,
         -0.31650537,  0.46054456],
        ...,
        [-0.09196634,  0.25417855, -0.13339621, ..., -0.01542251,
          0.06937581,  0.30769506],
        [-0.16548762,  0.26996222, -0.19250244, ..., -0.00263452,
          0.01036316,  0.21397181],
        [-0.03401021,  0.38231167, -0.17483445, ...,  0.10709681,
         -0.30617112,  0.23739932]], dtype=float32),
 array([[-0.01060541,  0.36867306,  0.19885413, ...,  0.71926856,
          0.505248  ,  0.392467  ],
        [-0.10582381,  0.38805786,  0.22003515, ...,  0.5171252 ,
          0.3589728 ,  0.04126201],
        [ 0.02648921,  0.09289545,  0.34511152, ...,  0.21367927,
          0.16508472,  0.14857253],
        ...,
        [-0.18316106, -0.16621496,  0.0

In [16]:
ngrams[0][:5], ngram_vecs[0][:5]

(['john was walking',
  'was walking down',
  'walking down the',
  'down the street',
  'the street when'],
 array([[-0.0140322 ,  0.00529452,  0.09398252, ..., -0.12720276,
         -0.18071574,  0.15556256],
        [ 0.2069744 ,  0.17052889,  0.19572538, ..., -0.2820218 ,
         -0.16869394,  0.28693688],
        [ 0.25348058,  0.16271347,  0.22691195, ..., -0.3728949 ,
         -0.31650537,  0.46054456],
        [ 0.26194477,  0.20343904, -0.097513  , ..., -0.31586525,
         -0.12883197,  0.26321927],
        [ 0.23764928,  0.32365665, -0.25521722, ...,  0.2341072 ,
         -0.15450497,  0.40597466]], dtype=float32))

In [18]:
queries = ng.encode_queries(["walked down the street", "bought from the shopkeeper"], amp=False)
queries

Encoding:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting:   0%|          | 0/2 [00:00<?, ?it/s]

array([[ 4.03001189e-01,  3.81821603e-01,  6.94982708e-03,
        -1.92269176e-01,  1.52531773e-01, -9.64645371e-02,
         2.67109483e-01,  2.03985617e-01,  1.63837485e-02,
        -3.54431391e-01,  2.48268574e-01,  2.53330052e-01,
        -2.76718773e-02,  8.27899575e-03, -1.71435103e-01,
        -1.89433157e-01,  1.06366172e-01,  1.75932482e-01,
        -1.48258537e-01,  2.46872365e-01, -1.52583987e-01,
        -1.81636170e-01,  1.46380320e-01, -1.63377091e-01,
         5.93268201e-02, -1.07949562e-01,  1.40100271e-01,
        -2.36185789e-02, -2.01877743e-01,  4.44605082e-01,
        -8.70370790e-02, -2.91440710e-02, -8.24845023e-03,
        -8.75794664e-02,  5.83132356e-02, -3.20260614e-01,
        -2.93552577e-01,  6.37668148e-02,  7.30239749e-02,
         3.04325849e-01, -2.00709626e-02, -1.23105541e-01,
         1.60420626e-01, -1.57648683e-01, -2.37811238e-01,
         2.15459555e-01,  7.64573365e-03,  6.86799824e-01,
        -5.68540096e-01,  5.49343452e-02,  4.12428267e-0

In [19]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, metric="cosine")
nn.fit(ngram_vecs[0])
dists, idx = nn.kneighbors(queries)
[ngrams[0][i] for i in idx[1]]

['to buy something',
 'he wanted to buy something',
 'he wanted to buy',
 'the vendor started to',
 'buy something. his',
 'vendor started to',
 'to buy something, but his',
 'he wanted to buy something,',
 'to buy something,',
 'buy something, but his']

In [21]:
[ngrams[0][i] for i in idx[0]]

['walking down the street',
 'was walking down the street',
 'walking down the',
 'walking down the street when',
 'john was walking down the street',
 'was walking down',
 'down the street',
 'walking down the street when he',
 'was walking down the street when',
 'was walking down the']

In [9]:
ngrams[0]

['john was walking',
 'was walking down',
 'walking down the',
 'down the street',
 'the street when',
 'street when he',
 'when he saw',
 'he saw a',
 'saw a vendor',
 'a vendor.',
 'vendor. he',
 '. he was',
 'he was so',
 'was so excited',
 'so excited!',
 'excited! he',
 '! he wanted',
 'he wanted to',
 'wanted to buy',
 'to buy something',
 'buy something,',
 'something, but',
 ', but his',
 'but his mommy',
 'his mommy said',
 'mommy said he',
 'said he had',
 'he had to',
 'had to wait',
 'to wait.',
 'wait. he',
 '. he watched',
 'he watched the',
 'watched the vendor',
 'the vendor with',
 'vendor with big',
 'with big eyes',
 'big eyes,',
 'eyes, but',
 ', but the',
 'but the vendor',
 'the vendor started',
 'vendor started to',
 'started to ignore',
 'to ignore him',
 'ignore him.',
 'him. john',
 '. john started',
 'john started to',
 'started to feel',
 'to feel sad',
 'feel sad until',
 'sad until he',
 'until he saw',
 'he saw the',
 'saw the modern',
 'the modern vendor